In [1]:
# Write out the problem statement
# Ask any clarifiying questions about that problem
# Re-state your understanding of the problem to the interviewer and confirm your understanding is correct
# Is my understanding of the problem correct?

In [ ]:
# Write a program that outputs the numbers 1-100
# If the number is a multiple of 3, output "Fizz"
# If the number is a multiple of 5, output "Buzz"
# If the number is a multiple of both 3 and 5, output "FizzBuzz"
# Otherwise, output the number itself

In [5]:
# do you want this a function?
def fizzbuzz(x):
    if x % 15 == 0:
        return "FizzBuzz"
    elif x % 5 == 0:
        return "Buzz"
    elif x % 3 == 0:
        return "Fizz"
    else:
        return x

# Example list comprehension
for i in range(1, 101):
    print(fizzbuzz(i))

1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14
FizzBuzz
16
17
Fizz
19
Buzz
Fizz
22
23
Fizz
Buzz
26
Fizz
28
29
FizzBuzz
31
32
Fizz
34
Buzz
Fizz
37
38
Fizz
Buzz
41
Fizz
43
44
FizzBuzz
46
47
Fizz
49
Buzz
Fizz
52
53
Fizz
Buzz
56
Fizz
58
59
FizzBuzz
61
62
Fizz
64
Buzz
Fizz
67
68
Fizz
Buzz
71
Fizz
73
74
FizzBuzz
76
77
Fizz
79
Buzz
Fizz
82
83
Fizz
Buzz
86
Fizz
88
89
FizzBuzz
91
92
Fizz
94
Buzz
Fizz
97
98
Fizz
Buzz


In [ ]:
# classifying fizzbuzz for 1-100 is the goal == we want to train on other numbers, then predict Y for 1-100

## Keras Workflow
- Setup training data
- Setup labels
- Encode the inputs
- Write a decoder for the outputs
- build a model, fit/train on training data
- Evaluate the model on test data
- We can use .predict_classes on out-of-sample data to make predictions

In [48]:
import warnings
warnings.filterwarnings('ignore')

# imports
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.models import Model

np.random.seed(23)

In [93]:
# specify the number of binary digits, variable so we can change it
num_digits = 12

In [94]:
# let's setup our training data
raw_training_data = np.random.randint(101, 2**num_digits, 2**num_digits - 102)
raw_training_data[0:3]

array([ 536, 3826, 2468])

In [95]:
len(np.arange(101, 1023))

922

In [96]:

raw_training_data.shape

(3994,)

In [97]:

# List out the state space. What are the different states that the data could be in:
# there's only 4 options/states in the FizzBuzz setup
# A number is a multiple of 3 and 5, only 3, only 5, or it ain't
# Now we'll need to one hot encode the training data for y
def fizz_buzz_encode(i):
    if i % 15 == 0:
        return np.array([0, 0, 0, 1]) # encoding for "fizzbuzz"
    elif i % 5 == 0: 
        return np.array([0, 0, 1, 0]) # encoding for "buzz"
    elif i % 3  == 0: 
        return np.array([0, 1, 0, 0]) # encoding for "fizz"
    else:
        return np.array([1, 0, 0, 0]) # encoding for the number output

In [98]:
## Binary Encoding function
# In order to binary encode the input values
def binary_encode(i, NUM_DIGITS):
    return np.array([i >> d & 1 for d in range(NUM_DIGITS)])

In [99]:
binary_encode(1, num_digits)

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [100]:
encoded_training_data = np.array([binary_encode(i, num_digits) for i in raw_training_data])

In [101]:
x_train = np.array(encoded_training_data)
x_train[0]

array([0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0])

In [102]:
print("Return only the number", fizz_buzz_encode(1))
print("Multiple of 3", fizz_buzz_encode(3))
print("Multiple of 5", fizz_buzz_encode(5))
print("Return only the number", fizz_buzz_encode(7))
print("Multiple of both 3 and 5", fizz_buzz_encode(15))

Return only the number [1 0 0 0]
Multiple of 3 [0 1 0 0]
Multiple of 5 [0 0 1 0]
Return only the number [1 0 0 0]
Multiple of both 3 and 5 [0 0 0 1]


In [103]:
# Y_train is the labels for our supervised algorithm
y_train = np.array([fizz_buzz_encode(i) for i in raw_training_data])

In [104]:
print(raw_training_data[0])
print(x_train[0])
print(y_train[0])

536
[0 0 0 1 1 0 0 0 0 1 0 0]
[1 0 0 0]


In [111]:
# convert from binary to fizzbuzz output
# decoder
def fizz_buzz(i, prediction):
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction]

In [106]:
model = Sequential()

In [107]:
model.add(Dense(1000, input_dim=num_digits, activation="relu"))
model.add(Dense(1000, activation="relu"))

# habit => set the number to be the number of possible outcomes
model.add(Dense(4, activation="softmax"))

In [108]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 1000)              13000     
_________________________________________________________________
dense_11 (Dense)             (None, 1000)              1001000   
_________________________________________________________________
dense_12 (Dense)             (None, 4)                 4004      
Total params: 1,018,004
Trainable params: 1,018,004
Non-trainable params: 0
_________________________________________________________________


In [109]:
model.compile(loss="categorical_crossentropy", optimizer="adagrad", metrics=["accuracy"])

In [110]:
model.fit(x_train, y_train, nb_epoch=30, batch_size=128)

Epoch 1/30
3994/3994 [==============================] - 1s 339us/step - loss: 1.3265 - accuracy: 0.5135
Epoch 2/30
3994/3994 [==============================] - 1s 165us/step - loss: 1.1309 - accuracy: 0.5333
Epoch 3/30
3994/3994 [==============================] - 1s 167us/step - loss: 1.1119 - accuracy: 0.5308
Epoch 4/30
3994/3994 [==============================] - 1s 179us/step - loss: 1.0907 - accuracy: 0.5371
Epoch 5/30
3994/3994 [==============================] - 1s 165us/step - loss: 1.0629 - accuracy: 0.5438
Epoch 6/30
3994/3994 [==============================] - 1s 180us/step - loss: 1.0283 - accuracy: 0.5613
Epoch 7/30
3994/3994 [==============================] - 1s 174us/step - loss: 0.9845 - accuracy: 0.5826
Epoch 8/30
3994/3994 [==============================] - 1s 169us/step - loss: 0.9351 - accuracy: 0.6077
Epoch 9/30
3994/3994 [==============================] - 1s 170us/step - loss: 0.8713 - accuracy: 0.6432
Epoch 10/30
3994/3994 [==============================] - 1s 163u

In [112]:
# fit model, yay! now what?
numbers = np.arange(1, 101)
x_test = np.transpose(binary_encode(numbers, num_digits))
x_test[0:2]

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [114]:
# neural net model usually has a variable name like "model" or "network"
y_test = model.predict_classes(x_test)

In [120]:
# function(argument)(another_set_of_arguments)
# b/c vectorize is returning a function
# predictions = np.vectorize(fizz_buzz)(numbers, y_test)
vectorized_fizz_buzz = np.vectorize(fizz_buzz)
predictions = vectorized_fizz_buzz(numbers, y_test)

In [121]:
predictions

array(['1', '2', 'fizz', '4', 'buzz', 'fizz', '7', '8', 'fizz', 'buzz',
       '11', 'fizz', '13', '14', 'fizzbuzz', '16', '17', 'fizz', '19',
       'buzz', 'fizz', '22', '23', '24', 'buzz', '26', 'fizz', '28', '29',
       'fizzbuzz', '31', '32', 'fizz', '34', 'buzz', 'fizz', '37', '38',
       'fizz', 'buzz', '41', 'fizz', '43', '44', 'fizzbuzz', '46', '47',
       'fizz', '49', 'buzz', 'fizz', '52', '53', 'fizz', 'buzz', '56',
       'fizz', '58', '59', '60', '61', '62', 'fizz', '64', 'buzz', 'fizz',
       '67', '68', 'fizz', 'buzz', '71', 'fizz', '73', '74', 'fizzbuzz',
       '76', '77', 'fizz', '79', 'buzz', 'fizz', '82', '83', 'fizz',
       'buzz', '86', 'fizz', '88', '89', 'fizzbuzz', '91', '92', 'fizz',
       '94', 'buzz', 'fizz', '97', '98', 'fizz', 'buzz'], dtype='<U8')

In [117]:
# correct answers for fizzbuzz on 1-100, these are our actual values
answer = np.array([])
for i in numbers:
    if i % 15 == 0: 
        answer = np.append(answer, "fizzbuzz")
    elif i % 5 == 0: 
        answer = np.append(answer, "buzz")
    elif i % 3 == 0: 
        answer = np.append(answer, "fizz")
    else: answer = np.append(answer, str(i))
print (answer)

['1' '2' 'fizz' '4' 'buzz' 'fizz' '7' '8' 'fizz' 'buzz' '11' 'fizz' '13'
 '14' 'fizzbuzz' '16' '17' 'fizz' '19' 'buzz' 'fizz' '22' '23' 'fizz'
 'buzz' '26' 'fizz' '28' '29' 'fizzbuzz' '31' '32' 'fizz' '34' 'buzz'
 'fizz' '37' '38' 'fizz' 'buzz' '41' 'fizz' '43' '44' 'fizzbuzz' '46' '47'
 'fizz' '49' 'buzz' 'fizz' '52' '53' 'fizz' 'buzz' '56' 'fizz' '58' '59'
 'fizzbuzz' '61' '62' 'fizz' '64' 'buzz' 'fizz' '67' '68' 'fizz' 'buzz'
 '71' 'fizz' '73' '74' 'fizzbuzz' '76' '77' 'fizz' '79' 'buzz' 'fizz' '82'
 '83' 'fizz' 'buzz' '86' 'fizz' '88' '89' 'fizzbuzz' '91' '92' 'fizz' '94'
 'buzz' 'fizz' '97' '98' 'fizz' 'buzz']


In [118]:
# Let's evaluate the model's predictions
evaluate = np.array(answer == predictions)
print(np.count_nonzero(evaluate == True) / 100)

0.98
